In [10]:
import numpy as np
import torchvision.datasets
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import os
from PIL import Image
import torch.nn.functional as F
cuda = torch.cuda.is_available()
cuda

True

In [11]:
%run MobileNetV2_2.py

def test(model, test_loader):
    model.eval()

    result = []
    for batch_num, (feats, labels) in enumerate(test_loader):
        feats, labels = feats.to(device), labels.to(device)
            

        outputs = model(feats)[1]
        
        _, pred_labels = torch.max(F.softmax(outputs, dim=1), 1)
        pred_labels_cpu = pred_labels.to('cpu')
        pred_labels_cpu = pred_labels_cpu.numpy()
        
        result.extend(pred_labels_cpu)
    return result

In [12]:
class ImageDataset(Dataset):
    def __init__(self, file_list, target_list):
        self.file_list = file_list
        self.target_list = target_list
        self.n_class = len(list(set(target_list)))

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img = Image.open(self.file_list[index])
        img = torchvision.transforms.ToTensor()(img)
        label = self.target_list[index]
        return img, label
def parse_data(datadir):
    img_list = []
    ID_list = []
    uniqueID_list = []
    for root, directories, filenames in os.walk(datadir):
        b = -1
        for filename in filenames:
            if filename.endswith('.jpg'):
                filei = os.path.join(root, filename)
                img_list.append(filei)
                ID_list.append(root.split('/')[-1])
                a = root.split('/')[-1]
                if a != b:
                    uniqueID_list.append(a)
                b = a

    # construct a dictionary, where key and value correspond to ID and target
    #uniqueID_list = list(set(ID_list))
    #print(uniqueID_list)
    class_n = len(uniqueID_list)
    target_dict = dict(zip(uniqueID_list, range(class_n))) ### True to 0 - 2299
    
    reverse_dict = dict(zip(range(class_n),uniqueID_list))
    label_list = [target_dict[ID_key] for ID_key in ID_list]

    print('{}\t\t{}\n{}\t\t{}'.format('#Images', '#Labels', len(img_list), len(set(label_list))))
    return img_list, label_list, class_n, reverse_dict

In [13]:
train_img_list, train_label_list, train_class_n, reverse_dict = parse_data('train_data/medium')
val_img_list, val_label_list, val_class_n, val_dict = parse_data('validation_classification/medium')
#test_img_list, test_label_list, test_class_n, _ = parse_data('test_c')

trainset = ImageDataset(train_img_list, train_label_list)
valset = ImageDataset(val_img_list, val_label_list)
#testset = ImageDataset(test_img_list, test_label_list)
testset = torchvision.datasets.ImageFolder(root='test_c/test_classification/', 
                                                       transform=torchvision.transforms.ToTensor())
test_dataloader = DataLoader(testset, batch_size = 100, shuffle = False, num_workers=3)

#Images		#Labels
822154		2300
#Images		#Labels
4601		2300


In [14]:
network = MobileNetV2(num_classes = 2300)
network.load_state_dict(torch.load('./model_epoch22.pth'))
device = 'cuda'
network.to('cuda')
network.eval()
result = test(network, test_dataloader)

In [15]:
result_a = []
for i in result:
    result_a.append(reverse_dict[i])

In [16]:
result_a = np.array(result_a)

In [17]:
print(result_a)
result_a = result_a.astype(int)

['947' '317' '1462' ... '807' '497' '674']


In [18]:
np.savetxt("hw2p2_2_model_22.csv", result_a, delimiter=",")

In [27]:
testsetv = torchvision.datasets.ImageFolder(root='test_v/', 
                                                       transform=torchvision.transforms.ToTensor())
testv_dataloader = DataLoader(testsetv, batch_size = 100, shuffle = False, num_workers=3)

In [28]:
testsetv[0][0]

tensor([[[0.3490, 0.3451, 0.3490,  ..., 0.1569, 0.1294, 0.2392],
         [0.3373, 0.3569, 0.3490,  ..., 0.1451, 0.1373, 0.2549],
         [0.3451, 0.3412, 0.3490,  ..., 0.1294, 0.1490, 0.2392],
         ...,
         [0.2784, 0.3098, 0.1608,  ..., 0.5412, 0.6000, 0.6000],
         [0.4706, 0.4745, 0.2431,  ..., 0.5961, 0.5647, 0.5176],
         [0.4588, 0.4941, 0.3333,  ..., 0.5451, 0.3765, 0.4784]],

        [[0.2588, 0.2549, 0.2549,  ..., 0.1294, 0.0863, 0.1647],
         [0.2471, 0.2627, 0.2549,  ..., 0.1176, 0.0902, 0.1765],
         [0.2471, 0.2392, 0.2471,  ..., 0.1020, 0.1020, 0.1608],
         ...,
         [0.1765, 0.2157, 0.0745,  ..., 0.3804, 0.4392, 0.4510],
         [0.3490, 0.3608, 0.1451,  ..., 0.4353, 0.4039, 0.3686],
         [0.3216, 0.3608, 0.2275,  ..., 0.3843, 0.2275, 0.3294]],

        [[0.3294, 0.3333, 0.3412,  ..., 0.1569, 0.1412, 0.2510],
         [0.3255, 0.3490, 0.3490,  ..., 0.1451, 0.1451, 0.2745],
         [0.3333, 0.3373, 0.3529,  ..., 0.1294, 0.1569, 0.

In [5]:
file = np.loadtxt('test_trials_verification_student.txt',dtype='str')

In [6]:
print(file)


[['299317.jpg' '248776.jpg']
 ['223110.jpg' '343700.jpg']
 ['226190.jpg' '258872.jpg']
 ...
 ['368216.jpg' '288465.jpg']
 ['365299.jpg' '240818.jpg']
 ['241731.jpg' '258859.jpg']]


MobileNetV2(
  (classifier): Sequential(
    (0): Dropout(p=0.2)
    (1): Linear(in_features=640, out_features=2300, bias=True)
  )
  (feature): Sequential(
    (0): Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU6(inplace)
    (3): InvertedResidual(
      (conv): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace)
        (3): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (4): InvertedResidual(
      (conv): Sequential(
        (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (

result_v = []
count = 0
index1 = []
index2 = []
for filename in file:
    index1_value =0
    index1_list = [int(i) for i in filename[0] if i.isdigit()] 
    for i in index1_list:
        pow = len(index1_list)-1
        index1_value += 10**pow * i
        index1.append(index1_value)
    index2_list = [int(i) for i in filename[1] if i.isdigit()]
    index2_value =0
    for i in index2_list:
        pow = len(index2_list)-1
        index2_value += 10**pow * i
        index2.append(index2_value)


In [24]:
img1_set = []
img2_set = []
for index in file:
        img1 = Image.open('test_v/test_verification/' + index[0])
        img1 = torchvision.transforms.ToTensor()(img1)
        img1_set.append(img1)

        img2 = Image.open('test_v/test_verification/' + index[1])
        img2 = torchvision.transforms.ToTensor()(img2)
        img2_set.append(img2)

KeyboardInterrupt: 

In [10]:
#result_v = []
i=0
n = 0
class TestImageDataset(Dataset):
    def __init__(self, file_list):
        self.file_list = file_list
        
    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img1 = Image.open('test_v/test_verification/' + self.file_list[index][0])
        img1 = torchvision.transforms.ToTensor()(img1)
        img2 = Image.open('test_v/test_verification/' + self.file_list[index][1])
        img2 = torchvision.transforms.ToTensor()(img2)
        return img1, img2

    


In [65]:
datasetv = TestImageDataset(file_list = file)

testv_dataloader = DataLoader(datasetv, batch_size = 100, shuffle = False, num_workers=3)


In [72]:
def test_v(model, test_loader):
    model.eval()
    result = np.array([])
    torch.no_grad()
    for batch_num, (feat1, feat2) in enumerate(test_loader):
        feat1, feat2 = feat1.to(device), feat2.to(device)
            

        output1 = model.feature(feat1)
        output1 = output1.mean([2,3])
        output2 = model.feature(feat2)
        output2 = output2.mean([2,3])

        
        emb1 = output1.to('cpu')
        emb1 = emb1.detach().numpy()
        emb2 = output2.to('cpu')
        emb2 = emb2.detach().numpy()
        
        #similarity = torch.dot(emb1, emb2)/(torch.norm(emb1)*torch.norm(emb2))
        result = np.append(result, np.sum(emb1*emb2,axis=1)/(np.linalg.norm(emb1, axis = 1)*np.linalg.norm(emb2, axis=1)))
        #print(result)
        if (len(result) % 10000 == 9999):
            print(len(result)+1)
        
        del output1
        del output2
        del emb1
        del emb2
        torch.cuda.empty_cache()
        
    return result


In [73]:
network = MobileNetV2(num_classes = 2300)


network.load_state_dict(torch.load('./model10_epoch20.pth'))
network.eval()
device = 'cuda'
network.to('cuda')
result_v = test_v(network, testv_dataloader)

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 14.73 GiB total capacity; 13.73 GiB already allocated; 15.88 MiB free; 279.63 MiB cached)

In [9]:
network = MobileNetV2(num_classes = 2300)


network.load_state_dict(torch.load('./model10_epoch20.pth'))
network.eval()
torch.no_grad()
device = 'cuda'
network.to('cuda')
result_v = []
i = 0
for index in file:
#index = file[0]
    i += 1
    if (i % 10000 == 9999):
        print(i+1)
    img1 = Image.open('test_v/test_verification/' + index[0])
    img1 = torchvision.transforms.ToTensor()(img1)
    img1 = torch.stack([img1], dim = 0)
    img1 = img1.to('cuda')

    img2 = Image.open('test_v/test_verification/' + index[1])
    img2 = torchvision.transforms.ToTensor()(img2)
    img2 = torch.stack([img2], dim = 0)
    img2 = img2.to('cuda')
    

    emb1 = network.feature(img1)[0]
    emb1 = emb1.mean([1,2])
    emb2 = network.feature(img2)[0]
    emb2 = emb2.mean([1,2])
#print(emb1)
#print(emb2)
    similarity = torch.dot(emb1, emb2)/(torch.norm(emb1)*torch.norm(emb2))
    similarity = similarity.to('cpu')
    similarity = similarity.detach().numpy()
    result_v.append(similarity)
    #result.append(similarity)
    #print(similarity)    
#result_v.append(similarity)
    del img1
    del img2
    del emb1
    del emb2
    torch.cuda.empty_cache()


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000


In [10]:
result_v = np.array(result_v)
np.savetxt("hw2p2_2_verification.csv", result_v, delimiter=",")

In [ ]:
img = Image.open('test_v/test_verification/299317.jpg')
img = torchvision.transforms.ToTensor()(img)

In [44]:
img

tensor([[[0.5725, 0.4118, 0.4431,  ..., 0.6980, 0.6902, 0.7020],
         [0.6157, 0.4392, 0.4431,  ..., 0.6667, 0.6941, 0.7098],
         [0.6745, 0.4941, 0.4863,  ..., 0.6588, 0.6745, 0.6745],
         ...,
         [0.2353, 0.1412, 0.2275,  ..., 0.4275, 0.4863, 0.4039],
         [0.4235, 0.3569, 0.3490,  ..., 0.5412, 0.5216, 0.4275],
         [0.4078, 0.3804, 0.3490,  ..., 0.5843, 0.4863, 0.4980]],

        [[0.4275, 0.2667, 0.2980,  ..., 0.5059, 0.4980, 0.5098],
         [0.4706, 0.2941, 0.2980,  ..., 0.4745, 0.5020, 0.5176],
         [0.5216, 0.3412, 0.3333,  ..., 0.4667, 0.4824, 0.4824],
         ...,
         [0.1608, 0.0549, 0.1255,  ..., 0.2941, 0.3725, 0.3020],
         [0.3569, 0.2706, 0.2353,  ..., 0.3922, 0.4000, 0.3137],
         [0.3373, 0.2902, 0.2235,  ..., 0.4235, 0.3529, 0.3765]],

        [[0.5882, 0.4275, 0.4588,  ..., 0.7255, 0.7176, 0.7294],
         [0.6314, 0.4549, 0.4588,  ..., 0.6941, 0.7216, 0.7373],
         [0.6941, 0.5137, 0.5059,  ..., 0.6824, 0.6980, 0.